/usr/local/bin/python3 -m pip install csv

In [13]:
import os
import csv
import unicodedata
import requests
import xml.etree.ElementTree

ourairports_base_url = 'https://davidmegginson.github.io/ourairports-data/'
ourairports_datafiles = ['airports.csv', 'countries.csv', 'regions.csv']
wikipedia_airline_codes_url = 'https://en.wikipedia.org/wiki/List_of_airline_codes'

image_path = '../images/airline_logos/'
   
download_logos = False

dump_airports = False
dump_airlines = False
dump_airplanes = False

### Download files from github.com (ourairports) and WikiPedia

In [14]:
for datafile in ourairports_datafiles:
    filename = '.' + datafile
    if (not os.path.isfile(filename)):
        print('Downloading ' + filename + '...')
        url = ourairports_base_url + datafile
        r = requests.get(url)
        if (r.status_code == 200):
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(r.text)
                file.close()

filename = '.airlines.html'
if (not os.path.isfile(filename)):
    print('Downloading ' + filename + '...')
    r = requests.get(wikipedia_airline_codes_url)
    if (r.status_code == 200):
        with open(filename,'w', encoding='utf-8') as file:
            file.write(r.text,)
            file.close()

In [15]:
if download_logos:
    for airline in airlines:
        logo_path = image_path + airline['icao'] + '.png'
        # Do not overwrite
        if os.path.exists(logo_path):
            continue

        logo_url = 'https://e0.flightcdn.com/images/airline_logos/90p/'+ airline['icao'] + '.png'
        #logo_url = 'https://flightaware.com/live/fleet/' + key + '.png'
        request = requests.get(logo_url, allow_redirects=True)
        if request.status_code == 200:
            with open(logo_path, 'wb') as file:
                file.write(request.content)
                file.close()
            print ('Downloaded ' + logo_url)

## Process files

In [16]:
countries = {}

# "id","code","name","continent","wikipedia_link","keywords"
with open('.countries.csv', 'r', encoding='utf-8') as csvfile:
    countries_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in countries_reader:
        countries[row['code']] = row

countries = sorted(countries.values(), key=lambda x: x['code'])

In [17]:
airports = {}
airports['QEK'] = {'iata_code': 'QEK', 'name': 'Budel/Kempen Airport', 'city': 'Budel', 'country': 'NL', 'region': 'NB', 'latitude': '51.2566454', 'longitude': '5.6016832', 'altitude': '114'}
airports['QEW'] = {'iata_code': 'QEW', 'name': 'Leicester Railway Station', 'city': 'Leicester', 'country': 'GB', 'region': 'ENG', 'latitude': '52.63965', 'longitude': '-1.14432', 'altitude': '276'}


# "id","ident","type","name","latitude_deg","longitude_deg","elevation_ft","continent","iso_country","iso_region","municipality","scheduled_service","gps_code","iata_code","local_code","home_link","wikipedia_link","keywords"
with open('.airports.csv', 'r', encoding='utf-8') as csvfile:
    airports_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in airports_reader:
        if len(row['iata_code']) == 3:
            airports[row['iata_code']] = row

airports = sorted(airports.values(), key=lambda x: x['iata_code'])

Prepare map for resolving countries to iso codes

In [18]:
countrymap = {}
for c in countries:
    countrymap[c['name']] = c['code']

# Incorrect mappings or typos
countrymap['Russian Federation'] = 'RU'
countrymap['Ivory Coast'] = 'CI'
countrymap['Global'] = 'ZZ'
countrymap['Republic of Korea'] = 'KR'
countrymap['Macao'] = 'AS'
countrymap['Netherlands Antilles'] = 'NA'
countrymap['São Tomé and Príncipe'] = 'ST'
countrymap['Democratic Republic of the Congo'] = 'CD'
countrymap['Reunion'] = 'RE'
countrymap['Somali Republic'] = 'SO'
countrymap['The Gambia'] = 'GM'
countrymap['Peru'] = 'PE'
countrymap['Swaziland'] = 'SZ'
countrymap['Macedonia'] = 'MK'
countrymap['Myanmar']= 'MM'
countrymap['Democratic People\'s Republic of Korea'] = 'KP'
countrymap['Moldovoa']= 'MD'
countrymap['Burkino Faso'] = 'BF'
countrymap['Syrian Arab Republic'] = 'SY'
countrymap['CANADA'] = 'CA'
countrymap['Republic of the Congo'] = 'CG'
countrymap['Democratic Republic of Congo'] = 'CD'
countrymap['UAE'] = 'AE'
countrymap['Hong Kong SAR of China']= 'HK'
countrymap['French Guyana'] = 'GF'
countrymap['Lao People\'s Democratic Republic'] = 'LA'
countrymap['THAILAND'] = 'TH'
countrymap['Molodva'] = 'MD'
countrymap['the Netherlands'] = 'NL'
countrymap['Domocratic Republic of Congo'] = 'CD'
countrymap['Spain and France'] = 'ES'   # Level airline
countrymap['Sweden, Denmark and Norway'] = 'SE' #SAS
countrymap['Norway, Sweden and San Diego'] = 'GB' # SCAVAC flight school

In [19]:

root = xml.etree.ElementTree.parse('.airlines.html').getroot()

airlines=[]
# Missing airlines
airlines.append({'iata_code': 'AZ', 'icao_code': 'ITY', 'name': 'Italia Trasporto Aereo', 'callsign':'ITARROW', 'country': 'IT'})
airlines.append({'iata_code': 'DY', 'icao_code': 'NOZ', 'name': 'Norwegian Air Shuttle', 'callsign':'NORSEMAN', 'country': 'NO'})
airlines.append({'iata_code': '', 'icao_code': 'RHH', 'name': 'Redstar Aviation', 'callsign':'STAR CRESENT', 'country': 'TR'})
airlines.append({'iata_code': 'ZQ', 'icao_code': 'GER', 'name': 'German Airways', 'callsign':'GERMAN EAGLE', 'country': 'DE'})
airlines.append({'iata_code': '', 'icao_code': 'MMF', 'name': 'Multinational Multi-Role Tanker Transport Fleet', 'callsign':'MULTI', 'country': 'NL'})
airlines.append({'iata_code': '', 'icao_code': 'DFC', 'name': 'Aeropartner', 'callsign':'DARK BLUE', 'country': 'CZ'})
airlines.append({'iata_code': '', 'icao_code': 'EMM', 'name': 'Emperor Aviation', 'callsign':'RULER', 'country': 'MT'})
airlines.append({'iata_code': '', 'icao_code': 'NHX', 'name': 'Noordzee Helikopters Vlaanderen', 'callsign':'EVO', 'country': 'BE'})
airlines.append({'iata_code': 'JO', 'icao_code': 'JTD', 'name': 'Jet Time', 'callsign':'NEW JET', 'country': 'DK'})
airlines.append({'iata_code': '', 'icao_code': 'EUL', 'name': 'Euro Link GmbH', 'callsign':'EUROLINK', 'country': 'DE'})
airlines.append({'iata_code': 'ZF', 'icao_code': 'AZV', 'name': 'Azur Air', 'callsign':'AZUR AIR', 'country': 'RU'})
airlines.append({'iata_code': 'PS', 'icao_code': 'URO', 'name': 'European Cargo Limited', 'callsign':'EURO', 'country': 'GB'})
airlines.append({'iata_code': '', 'icao_code': 'PVD', 'name': 'PAD Aviation Service GmbH', 'callsign':'PAD AVIATION', 'country': 'DE'})
airlines.append({'iata_code': '', 'icao_code': 'LTC', 'name': 'SmartLynx Airlines', 'callsign':'LATCHARTER', 'country': 'LV'})
airlines.append({'iata_code': 'DK', 'icao_code': 'VKG', 'name': 'Sunclass Airlines', 'callsign':'VIKING', 'country': 'DK'})

# IATA	ICAO	Airline	Call sign	Country/Region	Comments
rows = root.findall('.//table//tr')
rows.pop(0)
fields = ['iata_code', 'icao_code', 'name', 'callsign', 'country', 'comments']
for row in rows:
    cells = row.findall('.//td')
    data = {}
    for i in range(0, len(fields)):
        if (len(list(cells)) < len(fields)):
            print('Error: ' + str(cells.count) + ' cells found in row ' + str(i))
            data['icao_code'] = ''
            break

        cell = cells[i]
        while True:
            val  = cell.text
            if val is not None:
                break
            # sometimes in a <a> tag, and sometimes in a <a><s></s></a>
            cell = cell.find('*')
            if cell is None:
                val = ''
                break

        #print(val)            
        data[fields[i]] = val.strip()

    if data['icao_code']:
        # Check defunct
        if (data['comments'] != None):
            if (data['comments'].lower().find('defunct') >= 0):
                continue
            if (data['comments'].lower().find('ceased') >= 0):
                continue
            if (data['comments'].lower().find('merged into') >= 0):
                continue
            if (data['comments'].lower().find('no longer current') >= 0):
                continue
            if (data['comments'].lower().find('renamed to') >= 0):
                continue
            if (data['comments'].lower().find('icao code no longer allocated') >= 0):
                continue

        # Exceptions in HTML
        if data['icao_code'] == '—' or data['icao_code'] == 'n/a' or data['country'] == 'n/a':
            continue
        # Country not found
        if not data['country'] in countrymap.keys():
            print('Country not found: ' + data['country'])
            continue

        data['iso_country'] = countrymap[data['country']]
        airlines.append(data)

airlines = sorted(airlines, key=lambda x: x['icao_code'])
#print(airlines)

Error: <built-in method count of list object at 0x7fa5a02c0d80> cells found in row 0
Error: <built-in method count of list object at 0x7fa59f58b4c0> cells found in row 0
Country not found: 
Country not found: 
Error: <built-in method count of list object at 0x7fa5a23606c0> cells found in row 0


## Write data

In [20]:
for country in countries:
      line = 'const country_t country_' + country['code'] + ' = { "' + country['code'] + '", "' + country['name']+ '", &country_flag_' + country['code'] + ' };'
      print(unicodedata.normalize('NFKD', line).encode('ascii', errors='ignore').decode('ascii'))


const country_t country_AD = { "AD", "Andorra", &country_flag_AD };
const country_t country_AE = { "AE", "United Arab Emirates", &country_flag_AE };
const country_t country_AF = { "AF", "Afghanistan", &country_flag_AF };
const country_t country_AG = { "AG", "Antigua and Barbuda", &country_flag_AG };
const country_t country_AI = { "AI", "Anguilla", &country_flag_AI };
const country_t country_AL = { "AL", "Albania", &country_flag_AL };
const country_t country_AM = { "AM", "Armenia", &country_flag_AM };
const country_t country_AO = { "AO", "Angola", &country_flag_AO };
const country_t country_AQ = { "AQ", "Antarctica", &country_flag_AQ };
const country_t country_AR = { "AR", "Argentina", &country_flag_AR };
const country_t country_AS = { "AS", "American Samoa", &country_flag_AS };
const country_t country_AT = { "AT", "Austria", &country_flag_AT };
const country_t country_AU = { "AU", "Australia", &country_flag_AU };
const country_t country_AW = { "AW", "Aruba", &country_flag_AW };
const c

In [21]:
for airport in airports:
    region = airport['iso_region'].split('-')[1]
    line = '{ "' + airport['iata_code'] + '", "' + airport['name'].replace('"', '\'') + '", "' + airport['municipality']+ '", "' + region + '", &country_' + airport['iso_country'] + ' ,' +  str(airport['latitude_deg']) + ', ' + str(airport['longitude_deg']) + ', ' + str(airport['elevation_ft'] + ' },')
    print(unicodedata.normalize('NFKD', line).encode('ascii', errors='ignore').decode('ascii'))

{ "AAA", "Anaa Airport", "Anaa", "U", &country_PF ,-17.3526, -145.509995, 10 },
{ "AAB", "Arrabury Airport", "Tanbar", "QLD", &country_AU ,-26.69639, 141.048718, 334 },
{ "AAC", "El Arish International Airport", "El Arish", "SIN", &country_EG ,31.073299408, 33.8358001709, 121 },
{ "AAD", "Adado Airport", "Adado", "GA", &country_SO ,6.095802, 46.6375, 1001 },
{ "AAE", "Rabah Bitat Airport", "Annaba", "23", &country_DZ ,36.826781, 7.81334, 16 },
{ "AAF", "Apalachicola Regional Airport", "Apalachicola", "FL", &country_US ,29.72750092, -85.02749634, 20 },
{ "AAG", "Avelino Vieira Airport", "Arapoti", "PR", &country_BR ,-24.103901, -49.789101, 2641 },
{ "AAH", "Aachen-Merzbruck Airport", "Aachen", "NW", &country_DE ,50.823055, 6.186389, 623 },
{ "AAI", "Arraias Airport", "Arraias", "TO", &country_BR ,-13.025154, -46.884107, 1923 },
{ "AAJ", "Cayana Airstrip", "Awaradam", "SI", &country_SR ,3.898681, -55.577907, 360 },
{ "AAK", "Aranuka Airport", "Buariki", "G", &country_KI ,0.185278, 173.63

In [22]:
for airline in airlines:
    logo_path = image_path + 'airline_logo_' + airline['icao_code'] + '.png'
    logo = '&airline_logo_' + airline['icao_code'] if os.path.exists(logo_path) else 'nullptr'

    values = ['"' + airline['icao_code'] + '"', '"' + airline['callsign'] + '"', '"' + airline['name'].replace('"', '\'') + '"', '&country_' + airline['iso_country'],  logo]
    line = "{ " + ", ".join(values) + "},"
    print(unicodedata.normalize('NFKD', line).encode('ascii', errors='ignore').decode('ascii'))


{ "AAB", "ABG", "Abelag Aviation", &country_BE, nullptr},
{ "AAC", "ARMYAIR", "Army Air Corps", &country_GB, nullptr},
{ "AAD", "Ambassador", "Mann Air Ltd", &country_GB, nullptr},
{ "AAF", "AIGLE AZUR", "Aigle Azur", &country_FR, nullptr},
{ "AAG", "ATLANTIC", "Atlantic Flight Training", &country_GB, nullptr},
{ "AAG", "ATLANTIC", "Air Atlantique", &country_GB, nullptr},
{ "AAH", "ALOHA", "Aloha Air Cargo", &country_US, &airline_logo_AAH},
{ "AAI", "BOREALIS", "Air Aurora", &country_US, nullptr},
{ "AAJ", "ALFA SUDAN", "Alfa Airlines", &country_SD, nullptr},
{ "AAK", "ALASKA ISLAND", "Alaska Island Air", &country_US, nullptr},
{ "AAL", "AMERICAN", "American Airlines", &country_US, &airline_logo_AAL},
{ "AAM", "", "Aim Air", &country_MD, &airline_logo_AAM},
{ "AAN", "AMSTEL", "Amsterdam Airlines", &country_NL, nullptr},
{ "AAO", "ATLANTIS AIR", "Atlantis Airlines (USA)", &country_US, nullptr},
{ "AAP", "ARABASCO", "Arabasco Air Services", &country_SA, nullptr},
{ "AAQ", "COPTERLINE", "

KeyError: 'iso_country'